In [ ]:
from binance.um_futures import UMFutures
import pandas as pd
import time

# --- API Configuration ---
# IMPORTANT: Replace with your own API keys
api_key = "YOUR_API_KEY"
api_secret = "YOUR_SECRET_KEY"

# Initialize Futures client
client = UMFutures(key=api_key, secret=api_secret)

# --- Parameters ---
symbol = "ETHUSDT"
start_date = "2024-08-10"
interval = "1d"  # Data interval (e.g., "15m", "1h", "1d")

print(f"Fetching {symbol} Futures kline data for {interval} interval...")

# Convert start date to timestamp
start_time = int(pd.Timestamp(start_date).timestamp() * 1000)
end_time = int(pd.Timestamp.now().timestamp() * 1000)

# List to hold all klines
all_klines = []

# Loop to fetch all data in batches
current_time = start_time
batch_count = 0

while current_time < end_time:
    batch_count += 1
    print(f"Fetching batch {batch_count}... (Starting from: {pd.to_datetime(current_time, unit='ms')})")
    
    # Fetch 1500 klines (Binance's max limit)
    klines = client.klines(
        symbol=symbol,
        interval=interval,
        startTime=current_time,
        limit=1500
    )
    
    if not klines:
        print("No more data received, ending loop.")
        break
    
    # Add to the list
    all_klines.extend(klines)
    
    # Update time for the next batch (1ms after the last kline's open time)
    # Using [0] (Open time) ensures we don't skip data
    current_time = klines[-1][0] + 1
    
    # Brief sleep to respect rate limits
    time.sleep(0.1)

print(f"Total {len(all_klines)} records fetched.")

# --- Data Processing ---

# Create DataFrame
df = pd.DataFrame(all_klines, columns=[
    'timestamp', 'open', 'high', 'low', 'close', 'volume', 
    'close_time', 'quote_asset_volume', 'number_of_trades', 
    'taker_buy_base_asset_volume', 'taker_buy_quote_asset_volume', 'ignore'
])

# Convert timestamp to datetime
df['timestamp'] = pd.to_datetime(df['timestamp'], unit='ms')

# Remove duplicate records just in case
df = df.drop_duplicates(subset=['timestamp'])

# Convert numeric columns to float
numeric_cols = ['open', 'high', 'low', 'close', 'volume', 'taker_buy_base_asset_volume', 'quote_asset_volume']
df[numeric_cols] = df[numeric_cols].astype(float)

# Select only desired columns
df_final = df[['timestamp', 'open', 'high', 'low', 'close', 'volume', 'taker_buy_base_asset_volume', 'quote_asset_volume']].copy()

# Rename Taker Buy Volume column
df_final.rename(columns={'taker_buy_base_asset_volume': 'taker_buy_volume'}, inplace=True)

# Sort by date
df_final = df_final.sort_values('timestamp')

# --- DATA QUALITY & MISSING DATA CHECK ---
print("\n=== DATA QUALITY CHECK ===")

# Calculate the expected time range
expected_start = pd.Timestamp(start_date).tz_localize(None) # Remove timezone info for comparison
expected_end = pd.Timestamp.now().floor(interval)     # Floor to the last completed interval

# Generate all expected timestamps based on the 'interval' variable
try:
    expected_timestamps = pd.date_range(
        start=expected_start, 
        end=expected_end, 
        freq=interval
    )
except ValueError as e:
    print(f"Error generating date range: {e}. Check if 'interval' is a valid pandas frequency.")
    expected_timestamps = pd.DatetimeIndex([])


print(f"Expected time range: {expected_start} - {expected_end}")
print(f"Expected total records: {len(expected_timestamps)}")
print(f"Fetched (unique) records: {len(df_final)}")

# Find missing timestamps
if not expected_timestamps.empty:
    actual_timestamps = set(df_final['timestamp'])
    expected_timestamps_set = set(expected_timestamps)
    missing_timestamps = sorted(expected_timestamps_set - actual_timestamps)
    
    if missing_timestamps:
        print(f"\n🚨 {len(missing_timestamps)} MISSING candlesticks detected!")
        print("\nFirst 10 missing timestamps:")
        for i, missing in enumerate(missing_timestamps[:10]):
            print(f"  {i+1}. {missing}")
        
        if len(missing_timestamps) > 10:
            print(f"  ... and {len(missing_timestamps) - 10} more")
        
        # Save missing timestamps to a CSV
        missing_df = pd.DataFrame({'missing_timestamp': missing_timestamps})
        missing_file = f"{symbol}_FUTURES_{interval.upper()}_MISSING.csv"
        missing_df.to_csv(missing_file, index=False)
        print(f"\nMissing timestamps saved to: {missing_file}")
        
    else:
        print("✅ No missing candlestick data found! All data is complete.")

    # --- Consecutive Gap Analysis ---
    print("\n=== CONSECUTIVE GAP ANALYSIS ===")
    
    if missing_timestamps:
        gaps = []
        current_gap_start = missing_timestamps[0]
        current_gap_end = missing_timestamps[0]
        
        # Create a Timedelta from the interval for comparison
        interval_delta = pd.to_timedelta(interval)
        
        for i in range(1, len(missing_timestamps)):
            # Check if the next missing timestamp is exactly one interval after the previous one
            if missing_timestamps[i] == missing_timestamps[i-1] + interval_delta:
                current_gap_end = missing_timestamps[i]
            else:
                # The gap has ended, record it
                gaps.append((current_gap_start, current_gap_end))
                # Start a new gap
                current_gap_start = missing_timestamps[i]
                current_gap_end = missing_timestamps[i]
        
        # Add the last gap
        gaps.append((current_gap_start, current_gap_end))
        
        print(f"Found {len(gaps)} total consecutive gap(s):")
        for i, (start, end) in enumerate(gaps[:5]): # Show first 5 gaps
            duration = (end - start)
            if start == end:
                print(f"  {i+1}. {start} (1 candlestick)")
            else:
                print(f"  {i+1}. {start} to {end} (Duration: {duration})")
        
        if len(gaps) > 5:
            print(f"  ... and {len(gaps) - 5} more gaps")

    # --- Data Quality Summary ---
    print("\n=== DATA QUALITY SUMMARY ===")
    if len(expected_timestamps) > 0:
        completeness = (len(df_final) / len(expected_timestamps)) * 100
        print(f"Data completeness: {completeness:.2f}%")
        
        if completeness >= 99:
            print("✅ Excellent data quality!")
        elif completeness >= 95:
            print("🟡 Good data quality")
        elif completeness >= 90:
            print("🟠 Medium data quality")
        else:
            print("🔴 Low data quality - use with caution!")
    else:
        print("Could not calculate completeness (no expected timestamps).")

# Duplicate check (should be 0 after drop_duplicates)
duplicates = df_final.duplicated(subset=['timestamp']).sum()
if duplicates > 0:
    print(f"🚨 {duplicates} duplicate records found (this shouldn't happen).")
else:
    print("✅ No duplicate records.")

# --- Save to CSV ---
output_file = f"{symbol}_FUTURES_{interval.upper()}_DATA.csv"
df_final.to_csv(output_file, index=False)

print(f"\n📁 Main data saved to: {output_file} ({len(df_final)} records)")
print(f"📊 Columns: {list(df_final.columns)}")

# Final data range info
if not df_final.empty:
    print(f"📅 Data range: {df_final['timestamp'].min()} to {df_final['timestamp'].max()}")

print("\n=== LAST 5 RECORDS ===")
print(df_final.tail())